In [0]:
# read the file from deltalake bronze 
df = spark.read.csv("/mnt/lendingclubdatalake/bronze/lending_club_2018/accepted_2007_to_2018Q4.csv", header=True)


In [0]:
print("=== ROW COUNT & SCHEMA ===")
print(f"Row count: {df.count()}")
df.printSchema()

=== ROW COUNT & SCHEMA ===
Row count: 2260701
root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: string (nullable = true)
 |-- funded_amnt: string (nullable = true)
 |-- funded_amnt_inv: string (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_sta

In [0]:
from pyspark.sql import functions as F

col = "id"

df.filter(~F.col(col).rlike("^[0-9]+$")).select(col).show(50, False)


+------------------------------------------------+
|id                                              |
+------------------------------------------------+
|Total amount funded in policy code 1: 6417608175|
|Total amount funded in policy code 2: 1944088810|
|Total amount funded in policy code 1: 1741781700|
|Total amount funded in policy code 2: 564202131 |
|Total amount funded in policy code 1: 1791201400|
|Total amount funded in policy code 2: 651669342 |
|Total amount funded in policy code 1: 1443412975|
|Total amount funded in policy code 2: 511988838 |
|Total amount funded in policy code 1: 2063142975|
|Total amount funded in policy code 2: 823319310 |
|Total amount funded in policy code 1: 1538432075|
|Total amount funded in policy code 2: 608903141 |
|Total amount funded in policy code 1: 2087217200|
|Total amount funded in policy code 2: 662815446 |
|Total amount funded in policy code 1: 3503840175|
|Total amount funded in policy code 2: 873652739 |
|Total amount funded in policy 

In [0]:
from pyspark.sql import functions as F

print(f"Total file rows: {df.count()}")

# Filter out malformed records prior to profiling
df_rows_filtered = df.filter(F.col("id").rlike("^[0-9.]+$"))

print(f"Filtered rows: {df_rows_filtered.count()}")


In [0]:
df_filtered.printSchema()

root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: string (nullable = true)
 |-- funded_amnt: string (nullable = true)
 |-- funded_amnt_inv: string (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string 

In [0]:
# obselete 
# 2. NULL COUNTS + NULL PERCENTAGES
from pyspark.sql import functions as F

#print("=== NULL COUNTS ===")
null_counts = df_filtered.select([F.count(F.when(F.col(c).isNull(), 1)).alias(c) for c in df_filtered.columns])
#display(null_counts)

#print("=== NULL PERCENTAGES ===")
total = df_filtered.count()
null_pct = null_counts.select([(F.col(c)/total).alias(c) for c in null_counts.columns])

null_pct_df_percent_rounded = null_pct.select([
    F.round(F.col(c), 8).alias(c) for c in null_pct.columns
])

#display(null_pct_df_percent_rounded)

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:190)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
columns_to_profile = [
    "id",
    "member_id",
    "funded_amnt",
    "funded_amnt_inv",
    "term",
    "int_rate",
    "installment",
    "grade",
    "sub_grade",
    "emp_title",
    "emp_length",
    "home_ownership",
    "annual_inc",
    "verification_status",
    "issue_d",
    "loan_status",
    "purpose",
    "title",
    "zip_code",
    "addr_state",
    "dti",
    "delinq_2yrs",
    "earliest_cr_line",
    "fico_range_low",
    "fico_range_high",
    "inq_last_6mths",
    "mths_since_last_delinq",
    "mths_since_last_record",
    "open_acc",
    "pub_rec",
    "revol_bal",
    "revol_util",
    "total_acc",
    "initial_list_status",
    "out_prncp",
    "out_prncp_inv",
    "total_pymnt",
    "total_pymnt_inv",
    "total_rec_prncp",
    "total_rec_int",
    "total_rec_late_fee",
    "recoveries",
    "collection_recovery_fee",
    "last_pymnt_amnt",
    "last_fico_range_high",
    "last_fico_range_low",
    "policy_code",
    "application_type"
]
df_profile = df_filtered.select(columns_to_profile)


In [0]:
# 2. NULL COUNTS + NULL PERCENTAGES
from pyspark.sql import functions as F

print("=== NULL COUNTS ===")
null_counts = df_profile.select([F.count(F.when(F.col(c).isNull(), 1)).alias(c) for c in df_profile.columns])
display(null_counts)

print("=== NULL PERCENTAGES ===")
total = df_profile.count()
null_pct = null_counts.select([(F.col(c)/total).alias(c) for c in null_counts.columns])

null_pct_df_percent_rounded = null_pct.select([
    F.round(F.col(c), 7).alias(c) for c in null_pct.columns
])

display(null_pct_df_percent_rounded)

=== NULL COUNTS ===


id,member_id,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,policy_code,application_type
0,2260668,0,0,0,0,0,0,0,166969,146907,0,4,0,0,0,1,23326,2,1,1712,30,30,1,1,31,1158341,1901321,110,92,54,1834,50,17,17,21,13,11,9,3,3,2,6,3,43,35,66,55


=== NULL PERCENTAGES ===


id,member_id,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,policy_code,application_type
0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0738583,0.0649839,0.0,1.8E-6,0.0,0.0,0.0,4.0E-7,0.0103182,9.0E-7,4.0E-7,7.573E-4,1.33E-5,1.33E-5,4.0E-7,4.0E-7,1.37E-5,0.5123888,0.8410439,4.87E-5,4.07E-5,2.39E-5,8.113E-4,2.21E-5,7.5E-6,7.5E-6,9.3E-6,5.8E-6,4.9E-6,4.0E-6,1.3E-6,1.3E-6,9.0E-7,2.7E-6,1.3E-6,1.9E-5,1.55E-5,2.92E-5,2.43E-5


In [0]:
# 3. DISTINCT COUNTS
print("=== DISTINCT COUNTS PER COLUMN ===")
distinct_counts = df_profile.agg(*(F.countDistinct(F.col(c)).alias(c) for c in df_profile.columns))
display(distinct_counts)

=== DISTINCT COUNTS PER COLUMN ===


id,member_id,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,policy_code,application_type
2260668,0,1572,10057,2,673,93301,7,35,512694,12,7,89369,4,140,10,268,63168,1175,270,11051,249,926,245,220,195,309,243,180,120,102286,1540,287,158,356239,368550,1633693,1311060,486576,636000,18530,132850,146314,704487,223,210,91,77


In [0]:
print("hi")

hi


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, DecimalType, StringType, DateType

column_types = {
    "id": StringType(),
    "member_id": StringType(),
    "funded_amnt": DecimalType(10,2),
    "funded_amnt_inv": DecimalType(10,2),
    "term": StringType(),
    "int_rate": DecimalType(10,2),
    "installment": DecimalType(10,2),
    "grade": StringType(),
    "sub_grade": StringType(),
    "emp_title": StringType(),
    "emp_length": StringType(),
    "home_ownership": StringType(),
    "annual_inc": DecimalType(10,2),
    "verification_status": StringType(),
    "issue_d": DateType(),
    "loan_status": StringType(),
    "purpose": StringType(),
    "title": StringType(),
    "zip_code": StringType(),
    "addr_state": StringType(),
    "dti": DecimalType(10,2),
    "delinq_2yrs": IntegerType(),
    "earliest_cr_line": DateType(),
    "fico_range_low": IntegerType(),
    "fico_range_high": IntegerType(),
    "inq_last_6mths": IntegerType(),
    "mths_since_last_delinq": IntegerType(),
    "mths_since_last_record": IntegerType(),
    "open_acc": IntegerType(),
    "pub_rec": IntegerType(),
    "revol_bal": DecimalType(10,2),
    "revol_util": DecimalType(10,2),
    "total_acc": IntegerType(),
    "initial_list_status": StringType(),
    "out_prncp": DecimalType(10,2),
    "out_prncp_inv": DecimalType(10,2),
    "total_pymnt": DecimalType(10,2),
    "total_pymnt_inv": DecimalType(10,2),
    "total_rec_prncp": DecimalType(10,2),
    "total_rec_int": DecimalType(10,2),
    "total_rec_late_fee": DecimalType(10,2),
    "recoveries": DecimalType(10,2),
    "collection_recovery_fee": DecimalType(10,2),
    "last_pymnt_amnt": DecimalType(10,2),
    "last_fico_range_high": IntegerType(),
    "last_fico_range_low": IntegerType(),
    "policy_code": StringType(),
    "application_type": StringType()
}

# Cast columns 
df_profile_newschema = df_profile
for col, dtype in column_types.items():
    if isinstance(dtype, DateType):
        df_profile_newschema = df_profile_newschema.withColumn(col, F.to_date(F.col(col), "MMM-yyyy"))
    elif isinstance(dtype, DecimalType):
        df_profile_newschema = df_profile_newschema.withColumn(col, F.col(col).cast(DecimalType(10,2)))
    elif isinstance(dtype, IntegerType):
        df_profile_newschema = df_profile_newschema.withColumn(col, F.col(col).cast(IntegerType()))
    else:  
        df_profile_newschema = df_profile_newschema.withColumn(col, F.col(col).cast(StringType()))



In [0]:
df_profile.printSchema()


root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- funded_amnt: string (nullable = true)
 |-- funded_amnt_inv: string (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- fico_range_low: string (nullable = true)

In [0]:

from pyspark.sql.types import StructType, StructField
new_schema = StructType([
    StructField("id", StringType(), nullable=False)
] + [StructField(c, dtype, nullable=True) for c, dtype in column_types.items() if c != "id"])

df_profile_newschema = spark.createDataFrame(df_profile_newschema.rdd, schema=new_schema)

# Verify schema
df_profile_newschema.printSchema()

root
 |-- id: string (nullable = false)
 |-- member_id: string (nullable = true)
 |-- funded_amnt: decimal(10,2) (nullable = true)
 |-- funded_amnt_inv: decimal(10,2) (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: decimal(10,2) (nullable = true)
 |-- installment: decimal(10,2) (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: decimal(10,2) (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: date (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: decimal(10,2) (nullable = true)
 |-- delinq_2yrs: integer (nullable = true)
 |-- earliest_cr_line: date (nullable = true)
 |-- 

In [0]:
df_profile_newschema.show(5)

---------------------------------------------------------------------------
NumberFormatException                     Traceback (most recent call last)
File <command-5887055519512469>, line 1
----> 1 df_profile_newschema.show(5)

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/classic/dataframe.py:305, in DataFrame.show(self, n, truncate, vertical)
    304 def show(self, n: int = 20, truncate: Union[bool, int] = True, vertical: bool = False) -> None:
--> 305     print(self._show_string(n, truncate, vertical))

File /databricks/spark/python/pyspark/sql/classic/dataframe.py:323, in DataFrame._show_string(self, n, truncate, vert

In [0]:
from pyspark.sql import functions as F

int_cols = [c for c, t in column_types.items() if isinstance(t, IntegerType)]

problem_columns = []
for c in int_cols:
    bad = df_profile.select(c).filter(~F.col(c).rlike("^[0-9]+$") & F.col(c).isNotNull())
    if bad.count() > 0:
        problem_columns.append(c)

problem_columns


['delinq_2yrs',
 'fico_range_low',
 'fico_range_high',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'total_acc',
 'last_fico_range_high',
 'last_fico_range_low']

In [0]:
df_profile.show(5)

+--------+---------+-----------+---------------+----------+--------+-----------+-----+---------+--------------------+----------+--------------+----------+-------------------+--------+-----------+------------------+------------------+--------+----------+-----+-----------+----------------+--------------+---------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+------------------+---------------+---------------+-------------+------------------+----------+-----------------------+---------------+--------------------+-------------------+-----------+----------------+
|      id|member_id|funded_amnt|funded_amnt_inv|      term|int_rate|installment|grade|sub_grade|           emp_title|emp_length|home_ownership|annual_inc|verification_status| issue_d|loan_status|           purpose|             title|zip_code|addr_state|  dti|delinq_2yrs|earliest_cr_line|fico_range_low|fico_range_high|inq

In [0]:
df.select("delinq_2yrs").distinct().show()


+-----------+
|delinq_2yrs|
+-----------+
|        1.0|
|       15.0|
|       17.0|
|        0.0|
|        9.0|
|       12.0|
|       18.0|
|       10.0|
|       14.0|
|        5.0|
|        6.0|
|        4.0|
|        7.0|
|      10.58|
|       11.0|
|        2.0|
|       30.0|
|        8.0|
|        3.0|
|       16.0|
+-----------+
only showing top 20 rows
